In [1]:
# Generate working reactions
import yaml, sys, os, random, re, glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sympy
sys.path.insert(0, '../atomic_SOC')
import chem_subs as chem
import multirx_subs as mrx

In [2]:
# Process all available molecular YAML files
moldata, Gdict = mrx.read_all_molec_yamls()

Reading 421 molecular YAML files
*** Warning: "No SOC data for open-shell linear molecule cch" in compute_E0()
*** Warning: "No SOC data for open-shell linear molecule ncn" in compute_E0()
*** Warning: "No SOC data for open-shell linear molecule nco" in compute_E0()
*** Warning: "No SOC data for open-shell linear molecule nh" in compute_E0()
*** Warning: "No SOC data for open-shell linear molecule o2" in compute_E0()


In [3]:
# Find any that have the same WebBook or ATcT name
molecs = list(moldata.keys())
Nmol = len(molecs)
for i in range(Nmol):
    m1 = molecs[i]
    try:
        wb1 = moldata[m1]['Identifiers']['names']['WebBook']
    except KeyError:
        wb1 = ''
    try:
        atct1 = moldata[m1]['Identifiers']['names']['ATcT']
    except KeyError:
        atct1 = ''
    for j in range(i):
        m2 = molecs[j]
        try:
            wb2 = moldata[m2]['Identifiers']['names']['WebBook']
        except KeyError:
            wb1 = ''
        try:
            atct2 = moldata[m2]['Identifiers']['names']['ATcT']
        except KeyError:
            atct2 = ''
        if m2 == m1:
            print(f'Duplicated molecule code {m1} = {m2}')
            continue
        if wb1 and (wb1 == wb2):
            print(f'{m1} and {m2} both have WebBook name {wb1}')
        if atct1 and (atct1 == atct2):
            print(f'{m1} and {m2} both have ATcT name {atct1}')           

chfchf and ch2cf2 both have ATcT name 1,1-Difluoroethene
hccn and hcccn both have ATcT name Cyanoacetylene
PrNH2 and iPrNH2 both have ATcT name 1-Propanamine


In [4]:
# Find any that are missing ATcT thermo data
for mol, mdat in moldata.items():
    try:
        eof0 = mdat['Refdata']['ATcT']['EoF0']
    except KeyError:
        eof0 = None
    try:
        eof298 = mdat['Refdata']['ATcT']['EoF298']
    except KeyError:
        eof298 = None
    if (eof0 is None) or (eof298 is None):
        print(f'\tNo ATcT thermo data for {mol}')
    try:
        wb298 = mdat['Refdata']['WebBook']['EoF298']
    except:
        wb298 = None
    try:
        loc298 = mdat['Refdata']['Local']['EoF298']
    except:
        loc298 = None
    if (eof298 is None) and (wb298 is None) and (loc298 is None):
        print(f'No 298K refdata at all for {mol}')

	No ATcT thermo data for acenaphthylene
	No ATcT thermo data for bicyc82one
	No ATcT thermo data for butadienyl2CCH
No 298K refdata at all for butadienyl2CCH
	No ATcT thermo data for butyrolactone
	No ATcT thermo data for ch3nch2
	No ATcT thermo data for ch3sh
	No ATcT thermo data for cme4
No 298K refdata at all for cme4
	No ATcT thermo data for cs
	No ATcT thermo data for cyc7
No 298K refdata at all for cyc7
	No ATcT thermo data for cycloheptene
No 298K refdata at all for cycloheptene
	No ATcT thermo data for cyrene
No 298K refdata at all for cyrene
	No ATcT thermo data for dioxolane
	No ATcT thermo data for DMF
	No ATcT thermo data for DMSO
	No ATcT thermo data for DMSO2
	No ATcT thermo data for EC
	No ATcT thermo data for etoc2h3
No 298K refdata at all for etoc2h3
	No ATcT thermo data for fluorene
	No ATcT thermo data for h2s
	No ATcT thermo data for ihexane
No 298K refdata at all for ihexane
	No ATcT thermo data for mC6H4Cl
No 298K refdata at all for mC6H4Cl
	No ATcT thermo data fo

In [5]:
# Get the name codes
dfnames = pd.read_csv(os.sep.join(['refdata', 'label_meanings.tsv']), sep='\t')

In [ ]:
Gdict['onno'].Avogadro()

In [ ]:
# Tabulate all functional groups
print('Identifying functional groups')
fungroup = {}  # key = molecule, value = dict (key = fgrp, value = list of tuple)
for molec, G in Gdict.items():
    fungroup[molec] = G.find_functional_group('all', spin=True)
    lname = moldata[molec]['Identifiers']['names']['local']
    print(f'{molec}  ({lname})')
    for grp, atoms in fungroup[molec].items():
        print(f'    {grp:<15s}  {atoms}')

In [ ]:
# Alternative dict with counts instead of atom tuples
fungroup_n = {}  # key = molec, value = dict (key = fgrp, value = count)
for molec, dfg in fungroup.items():
    fungroup_n[molec] = {k: len(v) for k, v in dfg.items()}

In [ ]:
# Get global frequency of all functional groups
fg_count = {}  # key = functional group, value = global count
for molec, dfg in fungroup_n.items():
    for fg, n in dfg.items():
        try:
            fg_count[fg] += n
        except KeyError:
            fg_count[fg] = n
# sort by increasing frequency
fg_count = {k: v for k, v in sorted(fg_count.items(), key=lambda item: item[1])}
print('Functional groups by rarity in dataset:')
fg_count

In [ ]:
def fgrp_rxn_counts(rxn):
    # Given a reaction (list of [molec, coeff] pairs),
    #   return a dict of functional groups
    #   key = f-group, value = net count
    fgcount = {}  # key = f-group, value = count
    for [mol, coeff] in rxn:
        gd = fungroup_n[mol]  # key = fgrp, value = count
        for fg, n in gd.items():
            try:
                fgcount[fg] += n * coeff
            except KeyError:
                fgcount[fg] = n * coeff
    return fgcount
def fgrp_are_balanced(rxn):
    # return True or False
    fgcount = fgrp_rxn_counts(rxn)
    bal = True
    for fg, n in fgcount.items():
        if n != 0:
            bal = False
    return bal

In [ ]:
def balance_rarest_fgrp(rxn):
    # Given a reaction (list of [molec, coeff] pairs) that is 
    #    unbalanced in functional groups, return a list of extended
    #    reactions that balance the rarest functional group
    fgnet = fgrp_rxn_counts(rxn)  # key = f-group, value = net count
    for fg in fg_count.keys():
        # fg_count is ordered by increasing frequency in the dataset
        net = fgnet.get(fg, 0)
        if net:
            break
    else:
        # all functional groups are balanced; just return the input
        #print(f'***Functional groups are balanced: {rxn}')
        return [rxn]
    # fg is the rarest unbalanced functional group
    #print(f'Eliminating functional group {fg} with coeff = {net}')
    rxlist = []
    educts = [pr[0] for pr in rxn]
    for molec, dfg in fungroup_n.items():
        # do not repeat any existing educt
        if molec in educts:
            continue
        if fg in dfg.keys():
            # it's a novel educt with the desired functional group
            # But does it include functional groups that are rarer than 'fg'?
            for grp in dfg.keys():
                if fg_count[grp] < fg_count[fg]:
                    # it does; do not use it
                    continue
            # OK, add this educt
            n = dfg[fg]
            coef = -net / n
            rx = rxn.copy()
            rx.append([molec, coef])
            rxlist.append(rx)
    return rxlist

In [ ]:
thresh_num_rx = 1e6  # stop searching if number of possible reactions exceeds this
thresh_grp_bal = 1000  # stop searching if number of functional-group-balanced reactions exceeds this
max_depth = 5  # maximum number of functional groups to balance

In [ ]:
target = 'phcooh'

In [ ]:
rxns = [ [[target, -1]] ]
print('Generating reactions that are balanced by functional group (only)')
for ifun in range(max_depth):
    print(f'search depth = {ifun+1}')
    old_rxns = rxns
    rxns = []
    for rx in old_rxns:
        rxl = balance_rarest_fgrp(rx)
        rxns += rxl
    nbal = sum([fgrp_are_balanced(rx) for rx in rxns])
    print(f'\t{len(rxns)} total reactions of which {nbal} are fg-balanced')
    if (nbal > thresh_grp_bal) or (len(rxns) > thresh_num_rx):
        # a threshold is exceeded
        print('Threshold exceeded--done')
        break
# Further consideration only for the fg-balanced reactions
rxns = [rx for rx in rxns if fgrp_are_balanced(rx)]

In [ ]:
# Get experimental EoF uncertainties, for use in culling reactions
#   also spin multiplicities, for the same purpose
#   also whether has WebBook thermo (same purpose)
uexp = {}
spinmult = {}
WBthermo = {}
for molec, mdata in moldata.items():
    u = 10  # use this if no uncertainty is available
    wb = 0
    if 'Refdata' in mdata.keys():
        for source, thermex in mdata['Refdata'].items():
            if 'EoF0' in thermex.keys():
                try:
                    ux = thermex['unc']
                    if ux == -999:
                        # defined quantity
                        ux = 0
                except KeyError:
                    pass
                try:
                    k = thermex['k_cover']
                except:
                    k = 2  # usual for thermo data
                u = min(u, ux/k)
            if source == 'WebBook':
                wb = 1
    uexp[molec] = u
    spinmult[molec] = mdata['Spin_mult']
    WBthermo[molec] = wb

In [ ]:
print('Aggressive culling so that no educt appears more than once')
# score each reaction; lower is better
scores = []
for rx in rxns:
    s = 0  # base score
    # penalty for free radicals
    smult = 0
    # penalty for lacking thermochem data in WebBook (maybe "weird" molecule)
    wb = 0
    for [educt, coeff] in rx:
        smult += max(1, abs(coeff)) * (spinmult[educt] - 1) ** 2
        if educt != target:
            wb += (1 - WBthermo[educt]) * max(1, abs(coeff))
    s += smult
    s += wb * wb
    # penalty for uncertainty in exptl thermochem
    uvec = np.array([uexp[mol] * coef for [mol, coef] in rx[1:]])
    uex = np.sqrt(np.dot(uvec, uvec))
    s += uex
    # penalty for lots of educts
    s += (len(rx) - 2) ** 2    
    # penalty for bare atoms
    nat = 0
    for [educt, coeff] in rx:
        if Gdict[educt].natom() == 1:
            nat += abs(coeff)
    s == nat * nat
    scores.append(s)
# which reactions to keep?
ikeep = []  # indices of reactions to keep
eused = []  # list of educts used
idx = np.argsort(scores)  # by increasing score
for irx in idx:
    neweduct = True
    rx = rxns[irx]
    for [educt, coef] in rx[1:]:  # skipping the target
        if educt in eused:
            neweduct = False
    if neweduct:
        # no repeated educts; add this reaction
        ikeep.append(irx)
        for [educt, coef] in rxns[irx][1:]:  # skipping the target
            eused.append(educt)
print(len(ikeep), 'reactions left')

In [ ]:
# Complete the reaction balancing using simple species (which may repeat among reactions)
simple = []  # list of "simple" molecules
scompos = []  # list of elemental composition dict
natom = []  # number of atoms in each molecule
exclude = ['o3']
for molec in moldata.keys():
    if molec == target:
        continue
    if molec in exclude:
        continue
    if (WBthermo[molec] != 1) and (uexp[molec] != 0):
        continue
    # precisely known thermochemistry
    if uexp[molec] > 0.1:  # kJ/mol
        continue
    # no more than three elements
    stoich = Gdict[molec].stoichiometry(asdict=True)
    nelem = len(stoich)
    if nelem > 3:
        continue
    # Add spin and charge to the dict, if nonzero
    ss = moldata[molec]['Spin_mult'] - 1
    if ss:
        stoich['2S'] = ss
    q = moldata[molec]['Charge']
    if q:
        stoich['charge'] = q
    simple.append(molec)
    scompos.append(stoich)
    natom.append(Gdict[molec].natom())
# sort the "simple" molecules by number of atoms
idx = np.argsort(natom)
natom = [natom[i] for i in idx]
simple = [simple[i] for i in idx]
scompos = [scompos[i] for i in idx]

In [ ]:
print(f'There are {len(simple)} "simple" molecules:')
for m, e in zip(simple, scompos):
    print(m, '\t', e)

In [ ]:
for i in ikeep:
    print(f'{scores[i]:.2f}')
    retval = mrx.check_reactions_balance(rxns[i], Gdict)
    print('>>', retval)
    if True in retval:
        print('\t\t', rxns[i])

In [ ]:
def reaction_net_spin_charge(rx, moldata):
    # 'rx' is list of [molec, coeff] pairs
    # 'moldata' is dict of YAML data for all molecules
    spin = 0  # 2*S
    charge = 0
    for [molec, coef] in rx:
        spin += (moldata[molec]['Spin_mult'] - 1) * coef
        charge += moldata[molec]['Charge'] * coef
    return spin, charge

def balance_using_simple(baldict, simple, scompos):
    # Balance a reaction using only "simple" educts
    # Do not use Gram-Schmidt 
    # 'baldict' {element: coefficient} that must be negated to balance a reaction
    #    "elements" can include '2S' (spin) and 'charge'
    # 'simple' is list of "simple" molecules, sorted by number of atoms
    # 'scompos' is list of dict of composition (incl. spin, charge) of the "simple"
    # Allow up to two educts to be used
    # Return a list of {molec: coeff} that negates 'baldict', sorted by 
    #    decreasing desirability (supposedly)
    balset = set(baldict.keys())
    elems = set()
    for comp in scompos:
        elems = elems.union(comp.keys())
    if not balset <= elems:
        # the list of simple molecules is missing essential elements
        chem.print_err('', f'elements {balset} are not spanned by {elems}')
    # construct composition vectors
    ordel = sorted(elems)
    #print('>>>ordel =\t', ordel)
    R = np.array([baldict.get(el, 0) for el in ordel])
    R = -R  # want to match the negative
    #print('>>>-R =\t\t', R)

    answers = []
    # Try one educt
    vec = []  # list of numpy array
    vn = []    # unit vectors 
    for comp in scompos:
        v = np.array([comp.get(el, 0) for el in ordel])
        vec.append(v)
        vn.append(chem.normalize(v))
    tol = 1.e-6
    for imol, x in enumerate(vn):
        p = np.dot(R, x)
        res = R - p * x
        if np.linalg.norm(res) < tol:
            # got an answer
            s = np.sign(p) * np.linalg.norm(R) / np.linalg.norm(vec[imol])
            answers.append({simple[imol]: s})
    if len(answers) == 0:
        # Try two educts
        nsimp = len(simple)
        for irow in range(nsimp):
            for el1 in scompos[irow].keys():
                if el1 not in baldict.keys():
                    continue
                # fix this one coefficient
                ia = ordel.index(el1)
                if R[ia] == 0:
                    continue
                a = R[ia] / vec[irow][ia]
                res = R - a * vec[irow]
                for icol in range(nsimp):
                    if np.linalg.norm(vn[icol] - vn[irow]) < tol:
                        # redundant (parallel) with first vector
                        continue
                    # is 'res' parallel to this vector?
                    p = np.dot(res, vn[icol])
                    res2 = res - p * vn[icol]
                    if np.linalg.norm(res2) < tol:
                        # got an answer
                        b = np.sign(p) * np.linalg.norm(res) / np.linalg.norm(vec[icol])
                        answers.append({simple[irow]: a, simple[icol]: b})
    if len(answers) == 0:
        # failed
        return None
    # remove any duplicates
    ans = []
    for d in answers:
        if d not in ans:
            ans.append(d)
    if len(ans) == 1:
        return ans
    # find the "best" answer (lowest score)
    score = [0] * len(ans)
    for ians, a in enumerate(ans):
        for educt in a.keys():
            imol = simple.index(educt)
            # disfavor reactions involving bare atoms
            if natom[imol] == 1:
                score[ians] += 3
            # disfavor radicals (spin != 0)
            compset = set(scompos[imol].keys())
            if '2S' in compset:
                score[ians] += 2
            # disfavor reactions that introduce new elements
            if not compset <= balset:
                score[ians] += 1
            # prefer fewer reactants
            score[ians] += len(a)
    #for s, r in zip(score, ans):
    #    print(f'::: score = {s} for ans =', r)
    idx = np.argsort(score)
    ans = [ans[i] for i in idx]
    return ans

In [ ]:
for i in ikeep.copy():
    retval = mrx.check_reactions_balance(rxns[i], Gdict, verbose=False, details=True)
    if retval[0][0] == False:
        # reaction needs balancing
        print(rxns[i])
        unbal = retval[1][0]  # dict with key = elem, value = coeff
        # add data about spin and charge
        sp, q = reaction_net_spin_charge(rxns[i], moldata)
        if sp:
            unbal['2S'] = sp
        if q:
            unbal['charge'] = q
        print('\t>>>>>>>>', unbal)
        # look for a "simple" molecule that contains only these elements
        small = balance_using_simple(unbal, simple, scompos)
        for d in small:
            print('\t<<', d)
        if small is None:
            chem.print_err('', f'Failed to balance reaction {rxns[i]}', halt=False)
            # remove it from the list
            ikeep.remove(i)
            continue
        if len(small) == 1:
            answer = small[0]
        elif len(small) > 1:
            # choose one solution; prefer any that use existing reactants
            rxset = set([r[0] for r in rxns[i]])
            solsets = [set(soln.keys()) for soln in small]
            dupl = []  # count of existing reactants used
            for solset in solsets:
                dupl.append(len(rxset.intersection(solset)))
            idx = np.argwhere(dupl).flatten()
            print(':::idx =', list(idx))
            if len(idx) == 1:
                # choose this one
                answer = small[idx[0]]
            elif len(idx) > 1:
                # choose the most overlap
                i = np.argmax(dupl)
                answer = small[i]
            else:
                # len(idx) = 0; just choose the first one
                answer = small[0]
        # add these educts to the reaction
        print('\t-----answer =', answer)
        for r, c in answer.items():
            for ie, [re, ce] in enumerate(rxns[i].copy()):
                if r == re:
                    # adjust coefficient of existing reactant
                    rxns[i][ie] = [re, ce + c]
                    break
            else:
                # a new reactant
                rxns[i].append([r, c])
        retval = mrx.check_reactions_balance(rxns[i], Gdict, verbose=False, details=False)
        print('\tafter balancing:', retval)

In [ ]:
# save the reactions generated by functional-group balancing
reactions = [rxns[i] for i in ikeep]
for rx in reactions:
    print(rx)

In [ ]:
rxn = mrx.reaction_bond_separation(target, Gdict[target])
print(rxn)

In [ ]:
for mol, stoich in mrx.BOND_SEP_PROTO_STOICHIOMETRIES.items():
    try:
        st = Gdict[mol].stoichiometry(asdict=True)
        if stoich == st:
            print(mol, 'is OK')
        else:
            print(f'Error: for {mol}', stoich, st)
    except KeyError:
        pass

# cruft below

In [ ]:
def reaction_net_spin_charge(rx, moldata):
    # 'rx' is list of [molec, coeff] pairs
    # 'moldata' is dict of YAML data for all molecules
    spin = 0  # 2*S
    charge = 0
    for [molec, coef] in rx:
        spin += (moldata[molec]['Spin_mult'] - 1) * coef
        charge += moldata[molec]['Charge'] * coef
    return spin, charge

def balance_using_simple(baldict, simple, scompos):
    # Balance a reaction using only "simple" educts
    # Uses Gram-Schmidt despite lack of orthonormality
    # 'baldict' {element: coefficient} that must be negated to balance a reaction
    #    "elements" can include '2S' (spin) and 'charge'
    # 'simple' is list of "simple" molecules, sorted by number of atoms
    # 'scompos' is list of dict of composition (incl. spin, charge) of the "simple"
    # Allow up to 3 educts to be used
    # Return {molec: coeff} that negates 'baldict'
    elems = set()
    for comp in scompos:
        elems = elems.union(comp.keys())
    if not set(baldict.keys()) <= elems:
        # the list of simple molecules is missing essential elements
        chem.print_err('', f'elements {set(baldict.keys())} are not spanned by {elems}')
    # construct composition vectors
    ordel = sorted(elems)
    print('>>>ordel =\t', ordel)
    R = np.array([baldict.get(el, 0) for el in ordel])
    R = -R  # want to match the negative
    print('>>>-R =\t\t', R)
    vecs = []  # list of numpy array
    vn = []    # unit vectors 
    for comp in scompos:
        v = np.array([comp.get(el, 0) for el in ordel])
        vecs.append(v)
        vn.append(chem.normalize(v))
    # Try one educt
    tol = 1.e-6
    resid = []
    for imol, x in enumerate(vn):
        res = R - np.dot(R, x) * x
        if simple[imol] == 'oh':
            print('<><> oh, R =', np.round(R, 3))
        if np.linalg.norm(res) < tol:
            # got an answer
            s = np.linalg.norm(R) / np.linalg.norm(vecs[imol])
            return {simple[imol]: s}
        resid.append(res)
    # we get here if one educt was not enough
    resid2 = np.zeros((len(simple), len(simple)), dtype=object)
    for irow, R1 in enumerate(resid):
        for imol, x in enumerate(vn):
            if imol == irow:
                continue
            res = R1 - np.dot(R1, x) * x
            if simple[irow] == 'oh':
                y = np.linalg.norm(res)
                z = np.linalg.norm(R1)
                print(f'<<< oh, |R1| = {z:.4f}, {simple[imol]} gives |res| = {y:.4f}')
            if np.linalg.norm(res) < tol:
                # got an answer
                s1 = np.dot(R, vn[irow]) / np.linalg.norm(vecs[irow])
                s2 = np.linalg.norm(R1) / np.linalg.norm(vecs[imol])
                print('<><>', {simple[irow]: s1, simple[imol]: s2})
            resid2[irow, imol] = res
    return None

In [ ]:

    
    
    matches = {}  # key = molec, value = multiplier
    natom = {}  # key = molec, value = number of atoms (sometimes needed)
    for mol, comp in zip(simple, scompos):
        if set(baldict.keys()) != set(comp.keys()):
            # wrong elements
            continue
        # look for same ratio of elements
        elems = sorted(comp.keys())
        nat = sum([v for k, v in comp.items() if k not in ['2S', 'charge']])
        natom[mol] = nat
        coefs = np.array([baldict[el] for el in elems])
        scoef = np.array([comp[el] for el in elems])
        relc = np.round(coefs / coefs[0], 4)  # ratios to be compensated
        rs = np.round(scoef / scoef[0], 4)  # ratios in simple molec
        if np.array_equal(relc, rs):
            # stoichiometric ratio is correct
            ratio = coefs[0] / scoef[0]
            print('\tMatch: ', mol, scoef, f'*{ratio}')
            matches[mol] = -ratio
            smallest = mol  # just to have an initial value
            minatom = natom[mol]
    if len(matches) > 0:
        # choose the smallest molecule
        for mol in matches.keys():
            if natom[mol] < minatom:
                minatom = natom[mol]
                smallest = mol
        return smallest, matches[smallest]
    # No match was found.  
    return None
    
def balance_retry(baldict, simple, scompos):
    # Try this when match_simple_educt() fails
    Can we first balance spin?
    sp = baldict.get('2S', 0)
    if sp == 0:
        print('\t>>>no spin to flip')
        return None
    # try flipping the spin
    flp = baldict.copy()
    flp['2S'] *= -1
    sm = match_simple_educt(flp, simple, scompos)
    if sm is None:
        print('\t>>>flipping spin did not help')
        return None
    # success; now reverse the coefficient
    sm = (sm[0], -sm[1])
    return sm

In [ ]:
for m, mdat in moldata.items():
    try:
        u = mdat['Refdata']['ATcT']['unc']
    except:
        u = -2
    if u < 0:
        print(m, u)

In [ ]:
moldata['ch3']['Refdata']

In [ ]:
rxns[4837]

In [ ]:
def reaction_net_spin_charge(rx, moldata):
    # 'rx' is list of [molec, coeff] pairs
    # 'moldata' is dict of YAML data for all molecules
    spin = 0  # 2*S
    charge = 0
    for [molec, coef] in rx:
        spin += (moldata[molec]['Spin_mult'] - 1) * coef
        charge += moldata[molec]['Charge'] * coef
    return spin, charge

def match_simple_educt(baldict, simple, scompos):
    # 'baldict' {element: coefficient} that must be compensated to balance a reaction
    #    two "elements" for this purpose can be '2S' (spin) and 'charge'
    # 'simple' is list of "simple" molecules
    # 'scompos' is dict of composition (incl. spin, charge) of the "simple"
    # Return (molec, coeff) with the "best" choice of simple molecule,
    #    and its coefficient
    matches = {}  # key = molec, value = multiplier
    natom = {}  # key = molec, value = number of atoms (sometimes needed)
    balset = set(baldict.keys())
    for mol, comp in zip(simple, scompos):
        if balset != set(comp.keys()):
            # wrong elements
            continue
        # look for same ratio of elements
        elems = sorted(comp.keys())
        nat = sum([v for k, v in comp.items() if k not in ['2S', 'charge']])
        natom[mol] = nat
        coefs = np.array([baldict[el] for el in elems])
        scoef = np.array([comp[el] for el in elems])
        relc = np.round(coefs / coefs[0], 4)  # ratios to be compensated
        rs = np.round(scoef / scoef[0], 4)  # ratios in simple molec
        if np.array_equal(relc, rs):
            # stoichiometric ratio is correct
            ratio = coefs[0] / scoef[0]
            print('\tMatch: ', mol, scoef, f'*{ratio}')
            matches[mol] = -ratio
            smallest = mol  # just to have an initial value
            minatom = natom[mol]
    if len(matches) > 0:
        # choose the smallest molecule
        for mol in matches.keys():
            if natom[mol] < minatom:
                minatom = natom[mol]
                smallest = mol
        return smallest, matches[smallest]
    # No match was found.  
    return None
    
def balance_retry(baldict, simple, scompos):
    # Try this when match_simple_educt() fails
    Can we first balance spin?
    sp = baldict.get('2S', 0)
    if sp == 0:
        print('\t>>>no spin to flip')
        return None
    # try flipping the spin
    flp = baldict.copy()
    flp['2S'] *= -1
    sm = match_simple_educt(flp, simple, scompos)
    if sm is None:
        print('\t>>>flipping spin did not help')
        return None
    # success; now reverse the coefficient
    sm = (sm[0], -sm[1])
    return sm

In [ ]:
targroups = list(fungroup[target].keys())
print('target =', target)
print(targroups)

In [ ]:
Nmatchgrps = {}  # key = molec name, value = dict of functional groups that match target (key = fgrp, value = count)
for molec, dgrp in fungroup.items():
    # includes the target molecule
    Nmatchgrps[molec] = {k: len(v) for k, v in dgrp.items() if k in fungroup[target].keys()}

Nmatchmols = {g: {} for g in targroups}  # key = functional group (in target), value = dict (key = molec, value = count)
for molec, dgrp in Nmatchgrps.items():
    for g, n in dgrp.items():
        if g in targroups:
            Nmatchmols[g][molec] = n

In [ ]:
print('Number of other molecules matching target functional groups:')
Nmols = {k: len(v) for k, v in Nmatchmols.items()}
# sort functional groups by count in dataset
targroups = [k for k, v in sorted(Nmols.items(), key=lambda item: item[1])]
for grp in targroups:
    print(f'{grp:<15s}  {len(Nmatchmols[grp]) - 1}')

In [ ]:
def gbalanced(gbal):
    # return True if functional groups are balanced
    # else False
    bal = True
    for g, n in gbal.items():
        if n:
            bal = False
    return bal

# start with the rarest functional group
fg = targroups[0]
# all possible neutralizers of this fgroup
rxns = []  # list of dict (key = molec, value = coeff)
gbals = []  # list of dict (key = fgroup, value = coeff)
for mol, nm in Nmatchmols[fg].items():
    # 'mol' is name of molecule, 'nm' is count of fgroup 'fg'
    if mol == target:
        continue
    rxn = {target: -1}
    gbal = {g: -n for g, n in Nmatchgrps[target].items()}  # key = fgroup, value = coeff
    c = -gbal[fg] / nm
    rxn[mol] = c
    for g, n in Nmatchgrps[mol].items():
        ng = gbal.get(g, 0)
        gbal[g] = ng + c * n
    print(rxn)
    print(gbal)

In [ ]:
Nmatchgrps

In [ ]:
target = 'ethanol'
#target = 'etono'
#target = 'dioxolane'
target = 'bicyc82one'
#target = 'c2h5cho'
#target = 'acoh'
#target = 'butyrolactone'
#target = 'acetoxyl'
#target = 'EC'
#target = 'onono2'
target = 'phcooh'
#target = 'naphthalene'
#target = 'PrNH2'
#target = 'fcn'
#target = 'AcNH2'
#target = 'DMSO'
#target = 'butadiene'
#target = 'benzyne'
#target = 'c6h5'
G = Gdict[target]
tstoich = G.stoichiometry(asdict=True)
tstoich

In [ ]:
print('Molecule =', target)
for grp, atoms in fungroup[target].items():
    if len(atoms):
        print(f'{grp:<15s}  {atoms}')

In [ ]:
# Find all other molecules with these functional groups
fgmatch = {}  # key = functional group, value = list of molec names
for grp in fungroup[target].keys():
    omolec = []
    for molec, fgroup in fungroup.items():
        if molec == target:
            continue
        match = fgroup.get(grp, [])
        if match:
            omolec.append(molec)
            matchgrps[molec].append(grp)
    fgmatch[grp] = omolec
print(f'Number of molecules that match functional groups in {target}:')
nmatch = {grp: len(omolec) for grp, omolec in fgmatch.items()}
# make list of functional groups by decreasing rarity
fgroups = [grp for grp, n in sorted(nmatch.items(), key=lambda item: item[1])]
for grp in fgroups:
    print(f'{grp:<15s}  {nmatch[grp]}')

In [ ]:
# Assign a score to each molecule, based upon functional-group matching
#   and rarity of functional groups
mscore = {mol: 0 for mol in Gdict.keys()} 
for grp, n in nmatch.items():
    s = 1. / n
    for m in fgmatch[grp]:
        mscore[m] += s
# sort by decreasing score
mscore = {m: s for m, s in sorted(mscore.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
mscore

In [ ]:
for m in mscore.keys():
    print(f'{m:<15s}  {matchgrps[m]}')

In [ ]:
# 

In [ ]:
print('You have to close Avogadro to continue using this notebook')
G.Avogadro()  # view molecule using Avogadro (separate window)

In [ ]:
Gdict['fluorene'].Avogadro()

In [ ]:
G.bonded_list()[4]

In [ ]:
bentype = G.Benson_atom_type()
bentype

In [ ]:
bentype = G.Benson_atom_type(detail=1)
bentype

In [ ]:
nbond = G.connection.sum(axis=0)
nbond

In [ ]:
df = G.list_bonds()
df

In [ ]:
subdf = df[df.order == 2]
display(subdf)
subdf = subdf[(subdf.el1 == 'C') & (subdf.el2 == 'C')]
display(subdf)
for irow, row in subdf.iterrows():
    i1 = row.i1; i2 = row.i2
    print(i1, bentype[i1], i2, bentype[i2])

In [ ]:
target = 'dioxolane'
rxns = mrx.generate_reactions(target, moldata, Gdict)
dfrx, fmtrx = mrx.build_reactions_DF(rxns, moldata, target)

In [ ]:
dfrx.style.format(fmtrx)

In [ ]:
crxns = mrx.cull_too_similar_reactions(target, rxns, moldata, Gdict, disjoint=True)
dftest, fmttest = mrx.build_reactions_DF(crxns, moldata, target)
dftest.sort_values('rho_E').style.format(fmttest)

In [ ]:
def print_delta(rxns):
    # just print stuff for this notebook
    global target, moldata
    # must be a list of reactions
    if not rxns:
        # empty list
        print('No reactions')
        return
    if not isinstance(rxns[0][0], list):
        rlist = [ rxns ]
    else:
        rlist = rxns
    # are the reactions balanced?
    oklist = mrx.check_reactions_balance(rlist, Gdict)
    okrx = [r for i, r in enumerate(rlist) if oklist[i]]
    # mrx.eq5_sums() returns dH/x0, not dH
    print(f'Of {len(rlist)} reactions, {len(okrx)} are balanced.')
    calcH, calcS = mrx.eq5_sums(okrx, target, moldata)
    for rx, H in zip(okrx, calcH):
        #print(rx)   # this is ugly
        lhs = []
        rhs = []
        for pair in rx:
            mol = pair[0]
            c = pair[1]
            if c < 0:
                # left side of equation
                # present coefficients as rational numbers
                crat = sympy.Rational(-c).limit_denominator(1000)
                cstr = f'{crat}'
                lhs.append('{:s} {:s}'.format(cstr, mol))
            else:
                # right side
                crat = sympy.Rational(c).limit_denominator(1000)
                cstr = f'{crat}'
                rhs.append('{:s} {:s}'.format(cstr, mol))
        lhs = ' + '.join(lhs)
        rhs = ' + '.join(rhs)
        print('{:s} = {:s}'.format(lhs, rhs))
        print('\tchange in calc. H = {:.1f} kJ/mol'.format(-H))  # assuming x0 = -1
    return

In [ ]:
G.find_benzene_rings()

In [ ]:
G.bondedatoms[0].values()

In [ ]:
G.bondlist

In [ ]:
G.element_indices('N')

In [ ]:
rxn = mrx.reaction_bond_separation(target, G)
print_delta(rxn)

In [ ]:
rxns = mrx.reaction_isomerization(target, Gdict)
print_delta(rxns)

In [ ]:
rxn = mrx.reaction_hydration(target, Gdict)
print_delta(rxn)

In [ ]:
rxn = mrx.reaction_hydrofluorination(target, Gdict)
print_delta(rxn)

In [ ]:
rxn = mrx.reaction_hydrochlorination(target, Gdict)
print_delta(rxn)

In [ ]:
rxn = mrx.reaction_to_elements(target, Gdict)
print_delta(rxn)

In [ ]:
rxn = mrx.reaction_hydrogenation(target, Gdict)
print_delta(rxn)

In [ ]:
rxn = mrx.reaction_oxygenation(target, Gdict)
print_delta(rxn)

In [ ]:
df = G.list_bonds()
df

In [ ]:
from sympy import nsimplify

def cull_to_disjoint_educts(rxns):
    # 'rxns' is a list of reactions, where a reaction is a list of [educt, coeff] pairs
    # reduce the list until no educt (besides the target) occurs in more than one reaction
    # the target is the first educt listed
    # Give preference to reactions with fewer educts, small integer coefficients
    nin = len(rxns)
    # score by coefficient ugliness
    ftol = 1.e-6
    rank = []
    for rxn in rxns:
        r = (len(rxn) - 1) * 2   # penalty for more educts
        for pair in rxn[1:]:
            denom = nsimplify(pair[1]).q
            if abs(denom) > 1 + ftol:
                # penalty for fraction (to encourage homologous reactions)
                r += denom
            # penalty for large coefficient (multiplies uncertainties)
            r += abs(pair[1])
        rank.append(float(r))
    idx = np.argsort(rank)
    # do the cull
    used = set()
    ikeep = []
    print('>>> lowest ranks:', np.array(rank)[idx[:5]])
    for i in idx:
        educts = set([pair[0] for pair in rxns[i][1:]])
        if educts.intersection(used) == set():
            # keep this reaction
            ikeep.append(i)
            used = used.union(educts)
    return [rxns[i] for i in ikeep]

In [ ]:
def Xtabulate_Benson_groups(target, Gdict, detail, commonG=False):
    # Return a DataFrame of Benson groups in all molecules that
    #   do not contain elements alien to the target molecule and
    #   (if 'commonG') that contain at least one B-group in the target
    # 'Gdict' is a dict of Geometry()
    # 'detail' specifies granularity
    tels = set(Gdict[target].stoichiometry(asdict=True).keys())
    tgrps = set(Gdict[target].Benson_groups(detail=detail, warn=False)[0])
    grps = []  # list of all groups
    compos = {}  # dict, key = molecule
    for mol, G in Gdict.items():
        els = set(G.stoichiometry(asdict=True).keys())
        if not els <= tels:
            # alien elements
            continue
        Bg = G.Benson_groups(asdict=True, detail=detail, warn=False)
        if commonG:
            if len(tgrps.intersection(set(Bg))) < 1:
                # no B-groups in common with the target
                continue
        compos[mol] = Bg
        for grp in Bg.keys():
            if grp not in grps:
                grps.append(grp)
    # build the DataFrame
    cols = ['molec'] + grps
    df = pd.DataFrame(columns=cols)
    row = [target] + [compos[target].get(grp, 0) for grp in grps]
    df.loc[0] = row  # put target at the top
    for mol, st in compos.items():
        if mol == target:
            # don't enter twice
            continue
        row = [mol] + [st.get(grp, 0) for grp in grps]
        df.loc[len(df)] = row
    return df

In [ ]:
def tabulate_Benson_groups(target, Gdict, detail):
    # Return a DataFrame of Benson groups in all molecules that
    #   could plausibly form a reaction for the target
    # 'Gdict' is a dict of Geometry()
    # 'detail' specifies granularity
    tels = set(Gdict[target].stoichiometry(asdict=True).keys())
    tgrps = set(Gdict[target].Benson_groups(detail=detail, warn=False)[0])
    grps = set()  # set of all groups (for creating DataFrame)
    compos = {}  # dict, key = molecule
    for mol, G in Gdict.items():
        els = set(G.stoichiometry(asdict=True).keys())
        if not els <= tels:
            # alien elements; discard this molecule
            continue
        Bg = G.Benson_groups(asdict=True, detail=detail, warn=False)
        compos[mol] = Bg
        bset = set(Bg.keys())
        grps = grps.union(bset)       
    # build the DataFrame 
    cols = ['molec'] + sorted(grps)
    df = pd.DataFrame(columns=cols)
    row = [target] + [compos[target].get(grp, 0) for grp in cols[1:]]
    df.loc[0] = row  # put target at the top
    freq = {}  # frequency of each B-group
    '''
    # add any molecules whose B-groups are a subset of the target's
    for mol, comp in compos.items():
        if mol == target:
            continue
        gs = set(comp.keys())
        if (len(gs) > 0) and (gs <= tgrps):
            row = [mol] + [comp.get(grp, 0) for grp in cols[1:]]
            df.loc[len(df)] = row
        # how common is each group?
        for g, n in comp.items():
            freq[g] = freq.get(g, 0) + n
    # which target groups are not found in the DF?
    missing = []
    for g in tgrps:
        n = np.count_nonzero(df[g])
        if n < 2:
            # only in the target
            missing.append(g)
    '''
    # add molecules that contain target groups 
    gset = tgrps.copy()  # set of currently used groups
    for mol, comp in compos.items():
        if mol in df['molec'].values:
            continue
        gs = set(comp.keys())
        if gs.intersection(tgrps):
            row = [mol] + [comp.get(grp, 0) for grp in cols[1:]]
            df.loc[len(df)] = row
            gset = gset.union(gs)
    # add molecules that include only the expanded set of groups
    for mol, comp in compos.items():
        if mol in df['molec'].values:
            continue
        gs = set(comp.keys())
        if (len(gs) > 0) and (gs <= gset):
            row = [mol] + [comp.get(grp, 0) for grp in cols[1:]]
            df.loc[len(df)] = row

    # make list of B-groups by rarity
    Brare = sorted(freq.keys(), key=lambda k: freq[k])
    # add molecules that share the target's rarest group
    for g0 in Brare:
        if g0 in tgrps:
            # the rarest group
            break
        
    # remove columns with all zeros
    df = df.loc[:, (df != 0).any(axis=0)]
    return df

In [ ]:
detail = 0
dfBen = tabulate_Benson_groups(target, Gdict, detail=detail)
dfBen

In [ ]:
import time
start = time.time()
rxns = mrx.solve_descriptor_reactions(dfBen, maxeduct=8, verbose=True)
end = time.time()
t = end - start
print('Time = {:.0f} s = {:.1f} min = {:.2f} hr'.format(t, t/60, t/3600))

In [ ]:
detail = 0
print('detail =', detail)
dfBgrp = mrx.educts_Benson_groups(target, Gdict, detail=detail, warn=False)
display(dfBgrp)

In [ ]:
Gdict['ch2ohoh'].Benson_groups()

In [ ]:
rxns = mrx.solve_descriptor_reactions(dfBgrp, verbose=True)

In [ ]:
rbal = mrx.discard_unbalanced_reactions(rxns, Gdict, verbose=True)
print_delta(rbal)

In [ ]:
detail = 1
dfBbonds = mrx.Benson_bonds_table(target, Gdict, detail=detail, warn=False)
display(dfBbonds)

In [ ]:
import time
start = time.time()
rxns = mrx.solve_descriptor_reactions(dfBbonds, verbose=True)
end = time.time()
t = end - start
print('Time = {:.0f} s = {:.1f} min = {:.2f} hr'.format(t, t/60, t/3600))

In [ ]:
rbal = mrx.discard_unbalanced_reactions(rxns, Gdict, tol=1.e-3, verbose=True)

In [ ]:
crx = cull_too_similar_reactions(target, rbal, moldata, Gdict, disjoint=True)
crx

In [ ]:
print_delta(rbal)

In [ ]:
def all_reactions_for_target(target, Gdict, verbose=False):
    # generate all reasonable reactions for the educt
    # 'Gdict' is a dict of Geometry() objects (dict key = name of molecule)
    # Return a list of reactions, where a reaction is a list of [educt, coeff] pairs
    st = Gdict[target].stoichiometry(asdict=True)
    tels = list(st.keys())  # chemical elements in the target molecule
    tset = set(tels)
    # build DataFrame of all molecules whose elements are a subset of those of target
    cols = ['molec'] + tels
    df = pd.DataFrame(columns=cols)
    df.loc[0] = [target] + [st[el] for el in tels]
    for molec, G in Gdict.items():
        if molec == target:
            continue
        st = G.stoichiometry(asdict=True)
        if set(st.keys()) <= tset:
            # include this molecule
            df.loc[len(df)] = [molec] + [st.get(el, 0) for el in tels]
    start = time.time()
    rxns = mrx.solve_descriptor_reactions(df, verbose=verbose)
    end = time.time()
    t = end - start
    print('Time = {:.0f} s = {:.1f} min = {:.2f} hr'.format(t, t/60, t/3600))    
    return rxns, df

In [ ]:
# TAKES VERY LONG TIME
#rxns, dfelem = all_reactions_for_target(target, Gdict, verbose=True)

In [ ]:
rbal = mrx.discard_unbalanced_reactions(rxns, Gdict, tol=1.e-3, verbose=True)

In [ ]:
print('Found {:d} reactions'.format(len(rxns)))

In [ ]:
'''
def build_reactions_DF(rxns, moldata, target, rho='rho_E'):
    # given a list of reactions, return a DataFrame
    # with computed and exptl thermo (T=0) for analysis
    
    # create the DataFrame
    exptl = mrx.select_expt(moldata, T=0)  # selected exptl data
    okrx = rxn_with_expt(rxns, target, exptl)   # useable reactions
    eq6sum, uexp = mrx.eq6_sums(okrx, target, exptl)  # exptl sums needed to compute EoF
    print('>>> eq5')
    calcH, calcS = mrx.eq5_sums(okrx, target, moldata)  # slow step
    print('>>> eof')
    eof = [s5 - s6 for s5, s6 in zip(calcH, eq6sum)]
    print('counts: eof = {:d}, calcH = {:d}, uexp = {:d}, okrx = {:d}'.format(len(eof),
                                    len(calcH), len(uexp), len(okrx)))
    dfrx = pd.DataFrame({f'EoF': eof, 'dH(rxn)': calcH, 'uexp': uexp,
                     'Reaction': [reaction_string(rxn) for rxn in okrx]})
    # get values of rho variants (list of tuples)
    print('>>> rho')
    rho_E, rho_T, rho_c = mrx.gather_rho(okrx, target, moldata) 
    dfrx['rho_E'] = rho_E
    dfrx['rho_T'] = rho_T
    dfrx['rho_c'] = rho_c
    #dfrx = dfrx.sort_values('EoF')
    #fmt = {col: '{:.1f}' for col in dfrx.columns}
    #fmt['Reaction'] = '{:s}'
    #display(dfrx.style.format(fmt))
    return dfrx
'''    
def process_reactionDF(target, rho, dfrx, verbose=False):
    # analyze the data in a DataFrame from build_reactions_DF()
    print(f'Target = {target}')
    print(f'Non-uniform weighting using {rho}:')
    print('>>> wmean')
    wmean, semw, rwmsx, a, b, change, niter = mrx.nonuniform_weighting(target, rho, dfrx,
                                                            verbose=verbose)
    # combine SEMw with u_exp
    print('\tSEMw = {:.1f}, uexp = {:.1f} kJ/mol'.format(semw, rwmsx))
    unc = np.sqrt(semw**2 + rwmsx**2)  # 'rwmsx' from eq. (13)
    print('\tEoF = ({:.1f} ± {:.1f}) kJ/mol (standard uncertainty)'.format(wmean, unc))
    return dfrx, wmean, unc

In [ ]:
dfrx, wmean, unc = process_reactions(rbal, moldata, target, verbose=True)

In [ ]:
# consider only a fraction of reactions with lowest rho
frac = 0.5
nf = int(frac * len(dfrx))
print(f'Keeping the {nf} reactions (of {len(dfrx)}) with lowest {rho}')
dfcore = dfrx.sort_values(rho).iloc[:nf].copy()

In [ ]:
plt.scatter(dfcore.rho_E, dfcore.rho_c, facecolors='none', edgecolors='b', alpha=0.1)
#plt.scatter(dfrx.rho_E, dfrx.rho_c, facecolors='none', edgecolors='b', alpha=0.1)
plt.xlabel('rho_E')
plt.ylabel('rho_c')
#plt.xlim([0, 1000])
#plt.ylim([0, 500])
plt.show()

In [ ]:
T = 0.  # temperature
exptl = mrx.select_expt(moldata, T)
okrx = rxn_with_expt(rxns, target, exptl)
eq6sum, uexp = mrx.eq6_sums(okrx, target, exptl)

In [ ]:
# Make DataFrame of reactions
pd.set_option('display.max_rows', 500)
calcH, calcS = mrx.eq5_sums(okrx, target, moldata)
eof = [s5 - s6 for s5, s6 in zip(calcH, eq6sum)]
neduct = max([len(rx) for rx in okrx])
cols = ['Target', 'x0']
for n in range(neduct-1):
    cols.extend([f'Educt{n+1}', f'x{n+1}'])
cols.append('{:s}H'.format(chem.DELTA))
dfrx = pd.DataFrame(columns=cols)
for rx, H in zip(rxns, calcH):
    row = []
    for ed in rx:
        row.extend(ed)
    while len(row) < len(cols) - 1:
        row.extend(['', 0])
    row.append(np.round(H, 1))
    dfrx.loc[len(dfrx)] = row
dfrx['rho_E'] = abs(dfrx[cols[-1]])
dfrx['EoF'] = eof
dfrx.sort_values('rho_E')

In [ ]:
hmin = dfrx[cols[-1]].min()
hmax = dfrx[cols[-1]].max()
print('Hmin, Hmax = {:.1f}, {:.1f}'.format(hmin, hmax))

In [ ]:
plt.hist(dfrx[cols[-1]], bins=100)
plt.xlim([-5000, 5000])
plt.xlabel('kJ/mol')
plt.ylabel('counts')
plt.show()

In [ ]:
plt.scatter(dfrx[cols[-1]], dfrx.EoF)
#plt.xlabel(r'$\rho_E$')
plt.xlabel(cols[-1])
plt.ylabel(r'$\Delta_fH$')
#plt.xlim([0, 1000])
#plt.ylim([-500, 500])
plt.show()

In [ ]:
# plot EoF vs cutoff
rhocut = np.linspace(10, 2010)
eof = [dfrx[dfrx.rho_E < cut].EoF.mean() for cut in rhocut]
plt.plot(rhocut, eof)
plt.xlabel('rho_E cutoff')
plt.ylabel('EoF(target)')
plt.show()

In [ ]:
rho = 'rho_E'
print(f'Target = {target}')
print(f'Non-uniform weighting using {rho}:')
wmean, semw, rwmsx, a, b, change, niter = mrx.nonuniform_weighting(target, rho, dfrx,
                                                        verbose=False)
# combine SEMw with u_exp
print('\tSEMw = {:.1f}, uexp = {:.1f} kJ/mol'.format(semw, rwmsx))
unc = np.sqrt(semw**2 + rwmsx**2)  # 'rwmsx' from eq. (13)
print('\tEoF({:.1f} K) = ({:.1f} ± {:.1f}) kJ/mol (standard uncertainty)'.format(T, wmean, unc))

In [ ]:
eq6sum

In [ ]:
i = 1
print(rxns[i])
print_delta(rxns[i])

In [ ]:
def rxn_with_expt(rxin, target, exptl):
    '''
    Return a list of those reactions for which
      exptl thermo data are available.
    'rxns' is a list of reactions, where a reaction is
      a list of [educt, coeff] pairs.
    'target' is the name of the target molecule,
      not required to have exptl data
    'exptl' is a dict of exptl data for molecules
    '''
    rxns = []
    for rx in rxin:
        ok = True
        for pair in rx:
            molec = pair[0]
            try:
                eof = exptl[molec]['EoF']
                unc = exptl[molec]['unc']
            except KeyError:
                ok = False
        if ok:
            rxns.append(rx)
    return rxns